<a href="https://colab.research.google.com/github/sandikodev/Belajar-Diluar-Kelas/blob/main/Kecerdasan%5C%20Artifisial/a_Simple_LLM_Huggingface_Teori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Langkah 1**
> 📌 Instalasi Library Hugging Face untuk LLM di Colab

Sebelum menjalankan **LLM (Large Language Model)** di Colab, kita perlu menyiapkan beberapa library penting beserta dengan kegunaannya:

---

## 🔹 1. `transformers`
- **Peran**: Library utama dari Hugging Face.  
- **Fungsi**:  
  - Mengambil model dari Hugging Face Hub (`from_pretrained`)  
  - Menyediakan class **Tokenizer** & **Model** (misalnya `AutoTokenizer`, `AutoModel`)  
  - Dipakai untuk **text generation, summarization, translation**, dll.  
- **Analoginya**: *Buku panduan utama* untuk berbicara dengan model AI.

---

## 🔹 2. `accelerate`
- **Peran**: Optimizer agar model bisa berjalan di berbagai device (**CPU/GPU/TPU**).  
- **Fungsi**:  
  - Mengatur **device_map="auto"**  
  - Mendukung **multi-GPU** & **mixed precision (fp16/bf16)**  
- **Analoginya**: *Supir cerdas* yang tahu cara tercepat & hemat energi saat menjalankan mobil besar (LLM).

---

## 🔹 3. `bitsandbytes`
- **Peran**: Library untuk **quantization** (memadatkan model).  
- **Fungsi**:  
  - Menjalankan model besar di GPU kecil (misalnya 16 GB → bisa load model 30B).  
  - Mendukung mode `load_in_8bit=True` atau `load_in_4bit=True`.  
- **Analoginya**: *Mesin kompresi* yang membuat koper besar bisa masuk ke bagasi kecil 🚗.

---

## 🔹 4. `sentencepiece`
- **Peran**: Tokenizer berbasis subword (dibuat oleh Google).  
- **Fungsi**:  
  - Digunakan oleh model **T5, mBART, BERT multilingual, GPT multilingual**  
  - Mengubah teks → token → input untuk model.  
- **Analoginya**: *Kamus pemotong kata*, yang memecah kalimat jadi potongan kecil agar bisa dipahami mesin.

---

## ✅ Command Instalasi
Jalankan perintah berikut di Colab:

```python
!pip install transformers accelerate bitsandbytes sentencepiece
````

---

### 🧩 Visual Alur

1. **Kalimat** → (`sentencepiece`) → Token
2. **Token** → (`transformers`) → Model Hugging Face
3. **Model** → (`bitsandbytes`) → Lebih hemat memori
4. **Eksekusi** → (`accelerate`) → Optimasi CPU/GPU/TPU

# **Langkah 2**
> memuat model dan menjalankan tokenizer

artinya adalah mengambil model yang sudah dilatih sebelumnya (contoh: distilgpt2) lalu menyiapkan tokenizer supaya teks manusia bisa diubah menjadi token angka yang dipahami model.

---

### 🔹 Baris 1–2: Import Library

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
```

* **`transformers`** → library utama Hugging Face.

  * `AutoTokenizer` → otomatis pilih tokenizer sesuai model.
  * `AutoModelForCausalLM` → load model khusus untuk *Causal Language Modeling* (tebak kata berikutnya, cocok untuk text generation).
* **`torch`** → library PyTorch, dipakai untuk operasi tensor (angka-angka dalam model).

---

### 🔹 Pilih Nama Model

```python
model_name = "distilgpt2"
```

* `distilgpt2` adalah model GPT-2 versi ringan (hasil distillation).
* Dibuat lebih kecil dan cepat → ideal untuk belajar atau eksperimen di Colab.

---

### 🔹 Load Tokenizer

```python
tokenizer = AutoTokenizer.from_pretrained(model_name)
```

* Mengambil tokenizer dari Hugging Face Hub.
* Fungsi tokenizer = **memotong teks → token (angka)** yang bisa dipahami model.
* Contoh: `"saya suka makan"` → `[1234, 567, 890]`

---

### 🔹 Load Model

```python
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",   # otomatis ke GPU kalau ada
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)
```

* `from_pretrained(model_name)` → download & load bobot model dari Hugging Face.
* `device_map="auto"` → kalau ada GPU, model otomatis ditempatkan di GPU. Kalau tidak ada, tetap jalan di CPU.
* `torch_dtype` → tipe data tensor:

  * `float16` kalau ada GPU (lebih hemat memori).
  * `float32` kalau hanya CPU (standar).

---

### 🔑 Inti kode

Kode ini melakukan **setup awal** supaya kamu bisa:

1. **Mentransformasi teks** → token.
2. **Menjalankan model GPT-2 kecil** → prediksi kata berikutnya.
3. **Menghasilkan teks baru** dengan perintah `model.generate(...)`.

# **Langkah 3**
> memastikan model sudah termuat dengan benar

Mari kita bedah baris demi baris:

```python
from huggingface_hub import hf_hub_download
```

👉 Kita mengimpor fungsi `hf_hub_download` dari library **huggingface_hub**.
Fungsinya adalah **mengunduh file tertentu** dari repositori model di Hugging Face Hub (mirip seperti `git clone`, tapi hanya ambil file yang diperlukan).

---

```python
path = hf_hub_download(repo_id=model_name, filename="config.json")
```

👉 Di sini:

* `repo_id=model_name` → menunjuk ke nama model di Hugging Face (misalnya `"distilgpt2"`).
* `filename="config.json"` → file konfigurasi yang biasanya menyimpan detail arsitektur model (jumlah layer, ukuran hidden state, jumlah token, dll).

📌 Jadi perintah ini akan:

1. Cek apakah file `config.json` untuk model tersebut sudah ada di cache lokal.
2. Kalau belum ada → diunduh dari Hugging Face.
3. Kembalikan path (alamat) di mana file itu tersimpan di komputer.

---
📌 Contoh Kode:
```python
from huggingface_hub import hf_hub_download
import os

# Nama model yang dipakai
model_name = "distilgpt2"

# Unduh salah satu file model (config.json) untuk memastikan
path = hf_hub_download(repo_id=model_name, filename="config.json")

print("📂 File config.json disimpan di:", path)
print("📁 Folder model tersimpan di:", os.path.dirname(path))
```

👉 Menampilkan lokasi file `config.json` di sistem lokal, biar kita tahu **benar-benar sudah terunduh**.

---

📊 **Ringkasnya**:
Langkah 3 ini digunakan untuk **memastikan bahwa model (atau file pentingnya) sudah benar-benar terunduh dari Hugging Face Hub ke komputer kita**, bukan cuma “dipanggil” secara abstrak.

# **Langkah 4**
Mantap, kita masuk ke **langkah 4** ✨
Kode ini adalah bagian paling menarik, karena di sinilah **model mulai dipakai untuk menghasilkan teks (inference/text generation)**. Mari kita bedah pelan-pelan:

---

### 🚀 Langkah yang akan di kerjakan

Biasanya setelah model berhasil diload, kamu akan:

```python
# 1. Prompt awal
prompt = "Hari ini cuaca sangat cerah, saya ingin"

# 2. Ubah ke token
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# 3. Generate teks
outputs = model.generate(**inputs, max_new_tokens=50)

# 4. Decode ke kalimat
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```
> Alur: Teks → Tokenizer → Model → Output Token → Teks Lagi ?
---

## 📌 Penjelasan kode
### 1. Prompt
```python
prompt = "Hari ini saya belajar tentang komputer. Komputer terdiri dari"
```

👉 Ini adalah **teks awal** (seed / input) yang kita berikan ke model. Model akan melanjutkan kalimat ini sesuai dengan pengetahuan yang ia pelajari saat training.

---
### 2. Input
```python
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
```

👉 `tokenizer` mengubah teks jadi angka (**token IDs**) agar bisa dipahami model.

* `return_tensors="pt"` → hasil tokenisasi dikembalikan dalam format **PyTorch Tensor**.
* `.to(model.device)` → memastikan tensor dipindahkan ke perangkat tempat model berada (CPU atau GPU).

Analogi: **tokenizer bekerja seperti Google Translate, tapi dari bahasa manusia → MENJADI bahasa angka (dibaca: vektor)**.

---
### 3. Output
```python
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
)
```

👉 Di sini **model mulai menebak kelanjutan teks**.
Mari kita bedah parameter pentingnya:

* `max_new_tokens=100` → model boleh menambah **maksimal 100 token baru** setelah prompt.
* `do_sample=True` → model menggunakan sampling (acak terkendali), bukan sekadar memilih token probabilitas tertinggi.
* `top_k=50` → dari semua kemungkinan token, model hanya pilih **50 terbaik**.
* `top_p=0.95` → metode **nucleus sampling**: hanya ambil token yang akumulasi probabilitasnya mencapai 95%.
* `temperature=0.7` → mengatur seberapa **acak/kreatif** model.

  * Rendah (misal 0.2) → hasil lebih **deterministik** (aman tapi kaku).
  * Tinggi (misal 1.0) → hasil lebih **variatif/acak**.

Analogi:
🔮 Bayangkan model seperti orang yang diminta melanjutkan cerita:

* `max_new_tokens` = maksimal berapa kata/suku kata baru yang boleh ditambahkan.
* `top_k` = hanya memilih dari 50 kata kandidat teratas.
* `top_p` = hanya pilih dari kata-kata yang total peluangnya 95%.
* `temperature` = seberapa “berani/acak” orang ini dalam memilih kata.

---
### 4. Decode
```python
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```

👉 Model mengeluarkan output dalam bentuk token angka.

* `tokenizer.decode` mengubah angka itu kembali jadi teks manusia.
* `skip_special_tokens=True` artinya token khusus seperti `<pad>` atau `<eos>` tidak ditampilkan.

---

## ✨ Intinya:

Langkah 4 = **Menggunakan model untuk generate teks lanjutan dari sebuah prompt**.
Urutannya kira-kira:
> Alur: Teks → Tokenizer → Model → Output Token → Teks Lagi ?

**Prompt → Tokenizer → Model Generate → Decode → Output teks baru** 🎉

# Bonus pemahaman supaya makin mudah ngerti!
> apakah LLM ini juga sudah termasuk menerapkan Generative AI ? atau GAN ?


Pertanyaan bagus 👍

👉 **LLM (seperti GPT, distilGPT2, Flan-T5, dll.) termasuk bagian dari *Generative AI***, karena mereka bisa **menghasilkan konten baru** (teks, jawaban, cerita, kode, dll.) berdasarkan data latihannya.

Tapi ⚠️ **LLM tidak memakai GAN (Generative Adversarial Network)**.

* GAN punya dua komponen: **Generator** dan **Discriminator** yang saling “adu” (kompetisi). Biasanya dipakai untuk gambar realistis, suara, atau video palsu (deepfake).
* LLM memakai **Transformer (dengan mekanisme Attention)** untuk menebak token berikutnya dalam teks. Tidak ada “dua model yang berkompetisi” seperti di GAN.

📌 Jadi simpelnya:

* **LLM = Generative AI berbasis Transformer.**
* **GAN = Generative AI berbasis Generator vs Discriminator.**